In [33]:
import pandas as pd
import collections
import matplotlib.pyplot as plt

In [35]:
condition = input('Condition to test: ')

Condition to test: A22.0P5UM


In [36]:
all_phenotypes = pd.read_csv('/home/claudio/Comp_Genetics/results/bugwas_phenotypes.tsv', sep = '\t')
genotypes = pd.read_csv('/home/claudio/Comp_Genetics/raw_data/genotypes_noref_formatted.tsv', sep = '\t')

In [65]:
subset_phenotypes = all_phenotypes[all_phenotypes['pheno'].str.contains(condition)]
genotypes_1 = genotypes.rename(columns={"Unnamed: 0":"ps"})

In [66]:
ini_samples_genotypes = list(genotypes_1.columns.values[2:])
ini_samples_phenotypes = list(subset_phenotypes.iloc[:,0])

# Removing the reference sample from the phenotypes (aka NT12001)
ini_samples_phenotypes = [sample for sample in ini_samples_phenotypes if sample != 'NT12001']

In [81]:
# Which are the samples that do not match
nomatch_samples = [sample for sample in ini_samples_phenotypes if sample not in ini_samples_genotypes]
nomatch_samples = list(dict.fromkeys(nomatch_samples))
    
# Get rid of duplicates and nomatching samples on the phenotypes
phenotypes_1 = subset_phenotypes.drop_duplicates(subset=['ID'])
phenotypes_2 = phenotypes_1[~phenotypes_1['ID'].isin(nomatch_samples) & phenotypes_1['ID'].isin(ini_samples_phenotypes)] ## ~ = negation
    
# Get sample list to filter
samples_phenotypes = list(phenotypes_2.iloc[:,0])
samples_phenotypes.insert(0,'ps')
    
# Subset the genotype dataframe with the phenotype samples
subset_genotypes = genotypes_1[samples_phenotypes]
    
# Remove invariant sites
invariant_sites = []
for i, k in subset_genotypes.iterrows():
    if len(set(k[1:]))== 1:
        invariant_sites.append(False)
    else:
        invariant_sites.append(True)
subset_genotypes = subset_genotypes[invariant_sites]

In [86]:
!/home/claudio/Comp_Genetics/tassel-5-standalone/run_pipeline.pl -Xms512m -Xmx10g -importGuess /home/claudio/Comp_Genetics/input/snps.vcf.gz -CreateTreePlugin -clusteringMethod Neighbor_Joining -endPlugin -export -exportType SqrMatrix
!mv /home/claudio/Comp_Genetics/tassel-5-standalone/Matrix:snps.txt /home/claudio/Comp_Genetics/input/Distance_matrix.tsv

/home/claudio/Comp_Genetics/tassel-5-standalone/lib/ahocorasick-0.2.4.jar:/home/claudio/Comp_Genetics/tassel-5-standalone/lib/kotlinx-coroutines-core-1.3.0.jar:/home/claudio/Comp_Genetics/tassel-5-standalone/lib/kotlin-stdlib-1.3.50.jar:/home/claudio/Comp_Genetics/tassel-5-standalone/lib/jfreechart-1.0.19.jar:/home/claudio/Comp_Genetics/tassel-5-standalone/lib/slf4j-api-1.7.10.jar:/home/claudio/Comp_Genetics/tassel-5-standalone/lib/biojava-core-4.0.0.jar:/home/claudio/Comp_Genetics/tassel-5-standalone/lib/itextpdf-5.1.0.jar:/home/claudio/Comp_Genetics/tassel-5-standalone/lib/log4j-1.2.13.jar:/home/claudio/Comp_Genetics/tassel-5-standalone/lib/gs-core-1.3.jar:/home/claudio/Comp_Genetics/tassel-5-standalone/lib/snappy-java-1.1.1.6.jar:/home/claudio/Comp_Genetics/tassel-5-standalone/lib/fastutil-8.2.2.jar:/home/claudio/Comp_Genetics/tassel-5-standalone/lib/slf4j-simple-1.7.10.jar:/home/claudio/Comp_Genetics/tassel-5-standalone/lib/scala-library-2.10.1.jar:/home/claudio/Comp_Genetics/tasse

[ForkJoinPool.commonPool-worker-3] INFO net.maizegenetics.pipeline.TasselPipeline - net.maizegenetics.analysis.tree.CreateTreePlugin: time: Sep 16, 2019 18:08:46: progress: 70%
[ForkJoinPool.commonPool-worker-23] INFO net.maizegenetics.pipeline.TasselPipeline - net.maizegenetics.analysis.tree.CreateTreePlugin: time: Sep 16, 2019 18:08:46: progress: 80%
[ForkJoinPool.commonPool-worker-25] INFO net.maizegenetics.pipeline.TasselPipeline - net.maizegenetics.analysis.tree.CreateTreePlugin: time: Sep 16, 2019 18:08:46: progress: 90%
[ForkJoinPool.commonPool-worker-21] INFO net.maizegenetics.pipeline.TasselPipeline - net.maizegenetics.analysis.tree.CreateTreePlugin: time: Sep 16, 2019 18:08:46: progress: 100%
[pool-1-thread-1] INFO net.maizegenetics.analysis.distance.IBSDistanceMatrix3Alleles - IBSDistanceMatrix3Alleles: computeHetBitDistances time = 2 seconds
[pool-1-thread-1] INFO net.maizegenetics.plugindef.AbstractPlugin - Finished net.maizegenetics.analysis.tree.CreateTreePlugin: time: S

In [92]:
distance_matrix = pd.read_csv('/home/claudio/Comp_Genetics/input/Distance_matrix.tsv', sep = '\t', header = None)

In [93]:
distance_matrix

,0,1,2,3,4,5,6,7,8,9,...,686,687,688,689,690,691,692,693,694,695
0,NT12219,0.000000,0.016369,0.009429,0.010343,0.058500,0.011262,0.000680,0.070663,0.058651,...,0.057861,0.055464,0.041451,0.000519,0.057934,0.125000,0.010252,0.010522,0.010153,0.059328
1,NT12588,0.016369,0.000000,0.014328,0.015217,0.056194,0.017083,0.013521,0.076053,0.056446,...,0.056339,0.065170,0.038674,0.013868,0.056473,0.083333,0.016821,0.017205,0.015589,0.059486
2,NT12359,0.009429,0.014328,0.000000,0.009740,0.053117,0.010871,0.009328,0.081058,0.058294,...,0.053201,0.063365,0.032258,0.009290,0.053271,0.142857,0.010199,0.010225,0.009506,0.058116
3,NT12611,0.010343,0.015217,0.009740,0.000000,0.055359,0.009888,0.009260,0.080031,0.057796,...,0.055181,0.062853,0.038462,0.009600,0.055303,0.095238,0.010101,0.001640,0.000225,0.056974
4,NT12856,0.058500,0.056194,0.053117,0.055359,0.000000,0.061167,0.053473,0.060109,0.021639,...,0.000000,0.049782,0.003058,0.053902,0.000000,0.117647,0.059919,0.062591,0.058593,0.020034
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
690,NT12184,0.125000,0.083333,0.142857,0.095238,0.117647,0.050000,0.083333,0.066667,0.111111,...,0.117647,0.125000,0.000000,0.090909,0.117647,0.000000,0.150000,0.090909,0.047619,0.105263
691,NT12612,0.010252,0.016821,0.010199,0.010101,0.059919,0.011269,0.009738,0.070707,0.059038,...,0.059298,0.052690,0.032967,0.010199,0.059373,0.150000,0.000000,0.010512,0.009823,0.057306
692,NT12452,0.010522,0.017205,0.010225,0.001640,0.062591,0.010555,0.009252,0.089769,0.062809,...,0.063689,0.072474,0.038251,0.009536,0.063812,0.090909,0.010512,0.000000,0.001321,0.066126
693,NT12590,0.010153,0.015589,0.009506,0.000225,0.058593,0.009662,0.009038,0.079820,0.058861,...,0.058641,0.064128,0.038251,0.009152,0.058761,0.047619,0.009823,0.001321,0.000000,0.057711
